In [1]:
global_var = {

    # Connection
    'ip_address': '127.0.0.1',
    'port_nn_input': 8280,
    'port_pid_output': 8281,

    'buffer_size': 16,

    # Network 
    'input_dim': 4,
    'output_dim': 3,
    'hidden_dim': 10,
    'bias': False,
    'learning_rate': 0.001,
    'model_name': 'model.pt',

    #Train
    'batch_size': 32,
    'epochs': 10
}

class Color:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    MAGENTA = '\033[95m'
    CYAN = '\033[96m'
    WHITE = '\033[97m'
    RESET = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import struct
import time
import socket

# Utils

# Architecture

In [3]:
class FeedForwardNet(nn.Module):

    def __init__(self, input_dim, output_dim, hidden_dim, bias ) -> None:
        super(FeedForwardNet, self).__init__()

        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim =  output_dim

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

        self.layer_1 = nn.Linear(
            in_features = self.input_dim,
            out_features = self.hidden_dim,
            bias = bias
        )

        self.layer_2 = nn.Linear(
            in_features = self.hidden_dim,
            out_features = self.output_dim,
            bias = bias
        )
    
    def forward(self, x):
        
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)

        return x

In [4]:
class ControllerNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, lr, bias, model_name ) -> None:
        super(ControllerNetwork, self).__init__()

        self.network = FeedForwardNet(
            input_dim = input_dim,
            output_dim = output_dim,
            hidden_dim = hidden_dim,
            bias = bias
        )

        self.loss_fnc = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)

    def forward(self, x):
        network_output = self.network(x)
        return network_output
    
    def save(self, name):
        torch.save(self.state_dict(), name )
        print(f"Saved: {name}")

    def load(self, name):
        self.load_state_dict(torch.load(name) )
        print(f"loaded: {name}")

# PID

In [5]:
class PID():
    def __init__(self):
    
        self.error_1_step_back = 0
        self.error_2_step_back = 0
        self.threshold = 0.01   
        self.controller_output = 0

    
    def control(self, Kp, Ki, Kd, current_error):

        if current_error <= self.threshold:
            current_error = 0
            
        proportional_output = Kp * (current_error - self.error_1_step_back)
        integral_output = Ki * current_error
        differential_output = Kd * ( current_error - 2*self.error_1_step_back + self.error_2_step_back )

        delta_controller_output = proportional_output + integral_output + differential_output

        self.error_2_step_back = self.error_1_step_back
        self.error_1_step_back = current_error

        self.controller_output += delta_controller_output

        return self.controller_output

# Utils


In [6]:
def setup_receiver(ip_address, port_input):

    print(f"{Color.YELLOW}ip: {ip_address}, port_input: {port_input}{Color.RESET}")

    try:
        socket_nn_input = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        socket_nn_input.bind( (ip_address, port_input) )
        socket_nn_input.listen(1)

        print(f"{Color.BOLD}{Color.GREEN}Sockets listener created{Color.RESET}")

        return socket_nn_input
    except Exception as e:
        print(f"{Color.RED}An error occurred: {e}{Color.RESET}")

        if 'socket_nn_input' in locals():
            socket_nn_input.close()
            
        return None, None
    
def setup_sender(ip_address, socket_nn_input, port_output):

    print(f"{Color.YELLOW}ip: {ip_address}, port_output: {port_output}{Color.RESET}")

    try:
        
        sender = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sender.bind( (ip_address, port_output) )
        sender.listen(1)
        
        print(f"{Color.BOLD}{Color.GREEN}Sockets sender created{Color.RESET}")

        return  sender
    except Exception as e:
        print(f"{Color.RED}An error occurred: {e}{Color.RESET}")

        if 'socket_nn_input' in locals():
            socket_nn_input.close()
        if 'sender' in locals():
            sender.close()
        
        return None, None

def accept_connection(socket_server):
    connection, address = socket_server.accept()
    print(f"{Color.GREEN}Connection accepted!{Color.RESET}")
    return connection, address


def send_data(sender, message):
    try:
        message_to_send = struct.pack('!3d', *message)  # Pack one float
        sender.sendall(message_to_send)
        print(f"Sent float: {message}")

    except Exception as e:
        print(f"Error sending float: {e}")

def close_connections(socket_input, socket_output, receiver, sender):
    receiver.close()
    sender.close()
    socket_input.close()
    socket_output.close()

    print(f"{Color.GREEN}Sockets closed!")
    return


# Train

In [7]:
def train(receiver, sender, model, controller, epochs, batch_size, buffer_size):

    while True:
        data = receive_data(receiver, buffer_size)
        print(data)
        send_data(sender, 10.2)
        break
    '''
    for e in range(epochs):
        
        # collect at least batch_size sample
        for n_data in range(batch_size):
            data = receive_data(receiver)


        loss_value = 0

        for i, data in enumerate(dataset):

            # data: [reference_model_input, phi_d, phi_e,  phi]
            reference_model_input, phi_d, phi_e,  phi = data

            model.optmizer.zero_grad()
            
            pid_parameters = model(data)
            Kp, Ki, Kd = pid_parameters

            outputs = []
            for batch_index in range(batch_size):
                pid_output = controller.control( Kp[batch_index], Ki[batch_index], Kd[batch_index], phi_e[batch_index] ) 
                send_data(sender, pid_output)
                outputs.append(pid_output)
                
            loss = model.loss_fnc(torch.tensor(outputs), torch.tensor(phi_e) )
            loss.backward()

            model.optmizer.step()

            loss_value += loss.item()

    '''
    return 

In [8]:
network = ControllerNetwork(  input_dim=global_var['input_dim'],
                            output_dim=global_var['output_dim'],
                            hidden_dim=global_var['hidden_dim'],
                            bias=global_var['bias'],
                            lr=global_var['learning_rate'],
                            model_name=global_var['model_name']
                            )

pid_controller = PID()

In [9]:
def receive_data(connection, buffer_size, dim_input):
    
    expected_bytes = buffer_size  # Size for one double
    data = b''
    while len(data) < expected_bytes:
        more_data = connection.recv(expected_bytes - len(data))

        data += more_data
    data = list(struct.unpack(f'!{str(dim_input)}d', data))  # Unpack one double
    
    return data
   

# Set up the receiving server socket
receiver_socket = setup_receiver('127.0.0.1', 8280)
connection, addr = accept_connection(receiver_socket)
send_socket = setup_receiver('127.0.0.1', 8281)
connection2, addr = accept_connection(send_socket)

# Receive a message
data = receive_data(connection, 32, 4)
print(data)


send_data(connection2, [10.4, 11, 5.3])

ip: 127.0.0.1, port_input: 8280
Sockets listener created


In [ ]:
while True:
    continue

In [ ]:
socket_nn_input = setup_receiver(ip_address=global_var['ip_address'], 
                                port_input=global_var['port_nn_input'],
                                )

""" socket_pid_output = setup_sender(ip_address=global_var['ip_address'], 
                        port_output=global_var['port_pid_output'],
                        socket_nn_input=socket_nn_input
                        ) """

In [ ]:
receiver, address = accept_connection(socket_nn_input)
#sender, address = accept_connection(socket_pid_output)

In [ ]:
""" try:
    train(
        receiver=receiver,
        sender=sender,
        model=network,
        controller=pid_controller,
        epochs=global_var['epochs'],
        batch_size=global_var['batch_size'],
        buffer_size=4
    )
except Exception as e:
    print(f"{Color.RED}Error during training -> closing")
    print(e)
    close_connections(
        socket_input=socket_nn_input,
        socket_output=socket_pid_output,
        receiver=receiver,
        sender=sender
    )

close_connections(
        socket_input=socket_nn_input,
        socket_output=socket_pid_output,
        receiver=receiver,
        sender=sender
    ) """